# Molecular Simulation 
Carrying out molecular dynamics (MD) and Monte-Carlo (MC) simulations using a (trained) potential. 

In [1]:
from utils import set_env
# set_env('.env')

In [2]:
import os
os.environ["JAX_ENABLE_X64"] = "1"
os.environ["JAX_PLATFORM_NAME"] = "cpu" 
os.environ["XLA_PYTHON_CLIENT_PREALLOCATE"] = "false" 

In [3]:
from pantea.datasets import RunnerDataset
from pantea.simulation import (
    MDSimulator, 
    BrendsenThermostat, 
    MCSimulator,
    run_simulation
)
from pantea.atoms import Structure
from pantea.units import units as units


import matplotlib.pylab as plt
from pathlib import Path
from ase import Atoms
from ase.visualize import view
import ase.io
import jax.numpy as jnp

In [4]:
# from pantea.logger import set_logging_level
# import logging
# set_logging_level(logging.DEBUG)

from pantea.types import default_dtype
default_dtype.FLOATX=jnp.float64

base_dir = Path('./LJ')

## Data

In [ ]:
d = 6  # Angstrom
uc = Atoms('He', positions=[(d/2, d/2, d/2)], cell=(d, d, d))
s0 = Structure.from_ase(uc.repeat((7, 7, 7)))

# d = 10  # Angstrom
# uc = Atoms('Ar', positions=[(d/2, d/2, d/2)], cell=(d, d, d))
# s0 = Structure.create_from_ase(uc.repeat((7, 7, 7)))

atoms = s0.to_ase()
# view(atoms, viewer='ngl') # ase, ngl

## Potential

In [6]:
from pantea.types import Array
import jax
from functools import partial
from pantea.simulation import LJPotential

# He
ljpot = LJPotential(
    sigma=2.5238 * units.FROM_ANGSTROM,  # Bohr
    epsilon=4.7093e-04 * units.FROM_ELECTRON_VOLT,  # Hartree
    r_cutoff=6.3095 * units.FROM_ANGSTROM,  # 2.5 * sigma
)

# Ar
# ljpot = LJPotential(
#     sigma=3.405 * units.FROM_ANGSTROM,                       # Bohr
#     epsilon=0.01032439284 * units.FROM_ELECTRON_VOLT,        # Hartree
#     r_cutoff=8.5125 * units.FROM_ANGSTROM,                   # 2.5 * sigma
# )

# ljpot(s0), ljpot.compute_forces(s0)

## Molecular Dynamics (MD)

In [8]:
# v0 = MDSimulator.generate_random_velocity(temperature=300.0, mass=s0.mass, seed=2023)
# brendsen = BrendsenThermostat(target_temperature=300.0, time_constant=50.0 * units.FROM_FEMTO_SECOND)

md = MDSimulator(
    potential=ljpot,
    initial_structure=s0,
    time_step=0.5 * units.FROM_FEMTO_SECOND,
    temperature=300, # K
    # initial_velocity=v0,
    # thermostat=brendsen
)

In [9]:
md.get_pressure()

Array(6.51659918e-07, dtype=float64)

In [10]:
md.get_total_energy()

Array(0.48832374, dtype=float64)

In [11]:
# Warmp up
# run_simulation(md)

# %timeit run_simulation(md, num_steps=1, output_freq=-1)

In [12]:
run_simulation(md, num_steps=1000, output_freq=100) #, filename="md.xyz")

0          time[ps]:0.00000    Temp[K]:299.95012  Etot[Ha]:0.4883237353    Epot[Ha]:-0.0003923653   Pres[kb]:0.19173   
100        time[ps]:0.05000    Temp[K]:180.49172  Etot[Ha]:0.2937320422    Epot[Ha]:-0.0003475440   Pres[kb]:0.11535   
200        time[ps]:0.10000    Temp[K]:337.50049  Etot[Ha]:0.5495840454    Epot[Ha]:-0.0003137927   Pres[kb]:0.21572   
300        time[ps]:0.15000    Temp[K]:330.94523  Etot[Ha]:0.5389337659    Epot[Ha]:-0.0002834265   Pres[kb]:0.21154   
400        time[ps]:0.20000    Temp[K]:335.12906  Etot[Ha]:0.5457583650    Epot[Ha]:-0.0002756417   Pres[kb]:0.21418   
500        time[ps]:0.25000    Temp[K]:320.56006  Etot[Ha]:0.5220283124    Epot[Ha]:-0.0002680559   Pres[kb]:0.20486   
600        time[ps]:0.30000    Temp[K]:318.01530  Etot[Ha]:0.5178922473    Epot[Ha]:-0.0002578813   Pres[kb]:0.20325   
700        time[ps]:0.35000    Temp[K]:293.49149  Etot[Ha]:0.4779351878    Epot[Ha]:-0.0002577000   Pres[kb]:0.18758   
800        time[ps]:0.40000    Temp[K]:3

In [13]:
md.get_pressure() * units.TO_KILO_BAR

Array(0.19727315, dtype=float64)

In [14]:
md.positions * units.TO_ANGSTROM

Array([[ 1.19941342,  4.3296294 ,  2.70678353],
       [ 0.47843722,  2.07895327,  8.40255945],
       [ 3.77443508,  3.99753326, 13.52216957],
       ...,
       [36.71157116, 37.63130647, 23.95072796],
       [37.40653863, 40.52930075, 30.71223648],
       [37.86939568, 35.919512  , 38.93213743]], dtype=float64)

In [15]:
atoms = md.get_structure().to_ase()
# view(atoms, viewer='ngl') # ase, ngl

## Monte Carlo (MC)

In [16]:
mc = MCSimulator(
    potential=ljpot,
    initial_structure=s0,
    temperature=300, # K
    translate_step=0.3 * units.FROM_ANGSTROM,
    movements_per_step=10,
)

In [17]:
# Warmp up
# run_simulation(mc)

# %timeit run_simulation(mc, num_steps=1, output_freq=-1)

In [18]:
run_simulation(mc, num_steps=1000, output_freq=100, filename=Path("mc.xyz"))

0          Temp[K]:300.0000000000  Epot[Ha]:-0.0003923653   
100        Temp[K]:300.0000000000  Epot[Ha]:-0.0003693696   
200        Temp[K]:300.0000000000  Epot[Ha]:-0.0003808577   
300        Temp[K]:300.0000000000  Epot[Ha]:-0.0004226282   
400        Temp[K]:300.0000000000  Epot[Ha]:-0.0004656511   
500        Temp[K]:300.0000000000  Epot[Ha]:-0.0005302972   
600        Temp[K]:300.0000000000  Epot[Ha]:-0.0005892853   
700        Temp[K]:300.0000000000  Epot[Ha]:-0.0006245604   
800        Temp[K]:300.0000000000  Epot[Ha]:-0.0006825559   
900        Temp[K]:300.0000000000  Epot[Ha]:-0.0007941455   
1000       Temp[K]:300.0000000000  Epot[Ha]:-0.0008474004   


In [19]:
mc.positions * units.TO_ANGSTROM

Array([[ 2.34109917,  2.70732295,  3.65663507],
       [ 3.86856804,  2.18481293, 10.01926597],
       [ 4.23804463,  4.09003718, 14.46822375],
       ...,
       [38.59630648, 39.8119062 , 26.77745033],
       [38.5086862 , 37.05468303, 34.85910555],
       [40.55076128, 38.24893733, 40.75679957]], dtype=float64)

In [20]:
atoms = mc.get_structure().to_ase()
# view(atoms, viewer='ngl') # ase, ngl